In [1]:
import arcpy, os
from datetime import datetime, date
from homeless import *
import time
from arcgis import GIS
from arcgis.features import FeatureLayerCollection
import pandas as pd
gis = GIS('home')

In [2]:
arcpy.env.overwriteOutput = True
outpath = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\IllegalCampCoordination\Recieved'
path = outpath + '\\IllegalCampNotification_pro'
url = 'https://services5.arcgis.com/9s1YtFmLS0YTl10F/ArcGIS/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer'
camp_site = url + '/0'

In [3]:
listdates1 = []
cursor = arcpy.da.SearchCursor(camp_site, "Date")
for row in cursor:
    listdates1.append(row)

listdates = [dt[0] for dt in listdates1 if dt[0] is not None]    
maxdate = max(listdates)

In [4]:
maxdate

datetime.datetime(2023, 3, 6, 1, 16, 56, 716000)

In [5]:
# check whether there is a need to report
maxdatestr = str(maxdate)
datestr = maxdatestr.split('.')[0].split(' ')[0]
res = convert_date(datestr)
Y = res[1]
m = res[2]
d = res[3]
outfolder = os.path.join(outpath, Y, m+'_'+d)
outfolder_yr = os.path.join(outpath, Y)
filename = 'IllegalCampNotice_'+m+'_'+d+'_'+Y[2:4]+'.xlsx'
file = os.path.join(outfolder, filename)

In [6]:
flyrs = FeatureLayerCollection(url)
fl = flyrs.layers[0]
sdf = pd.DataFrame.spatial.from_layer(fl)
sdf['datestr'] = sdf.Date.apply(lambda x: str(x).split(' ')[0])
sdf_s = sdf[sdf.datestr == maxdatestr.split(' ')[0]]

In [32]:
df = convert_dtypes_arcgis(sdf_s)
df.drop(columns='datestr', inplace=True)

In [33]:
df.loc[:, 'Comments'] = df.Comments.apply(lambda x: trim_field_value(x))

In [26]:
len(df.Comments.values[0])

275

In [27]:
df.Comments.values[0][0:len(df.Comments.values[0])]

'Medium sized trailer full of garbage abandoned inside the gate to the Vida Transfer station and near the Leaburg Canal. An EWEB customer called to report the trailer on Sunday, EWEB PhySec department confirmed that EWEB does not own the property where the trailer is located.'

In [34]:
datacopy = df.spatial.to_featureclass(path + '\\MyProject4.gdb\\most_recent', overwrite=True)

In [30]:
def trim_field_value(x):
    return x[0:len(x)]

In [3]:
import geopandas as gpd
from homeless import *

In [5]:
outpath = r'\\clsrv111.int.lcog.org\GIS\projects\UtilityDistricts\eweb\DrinkingWater\IllegalCampCoordination\Recieved'
path = outpath + '\\IllegalCampNotification_pro'

In [6]:
dt = gpd.read_file(path + '\\MyProject4.gdb', layer= 'most_recent')

In [11]:
dt.date.values[0].split('T')[0]

'2023-03-06'

In [12]:
res = convert_date(dt.date.values[0].split('T')[0])

In [13]:
res

('3/6/2023', '2023', '3', '6')

In [1]:
import pandas as pd
import geopandas as gpd
import os
import requests
from bs4 import BeautifulSoup
from homeless import *
import openpyxl
from win32com.client import Dispatch

outpath = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\IllegalCampCoordination\Recieved'
path = outpath + '\\IllegalCampNotification_pro'

dat = pd.read_excel(path+'\\most_recent.xlsx')
cols2drop = ['OBJECTID', 'Join_Count', 'Unruly_inhabitants']
dat = dat.drop(cols2drop, axis=1)
dat.columns = list(map(lambda x: x.capitalize(), dat.columns))
dat.rename(columns={'Ownname': 'Owner_name', 'Addr1': 'Owner_address'}, inplace=True)
if 'Nearby_owner' in dat.columns:
    k = 2
    dat = dat[['Target_fid', 'Status', 'Comments', 'Date', 'Submitted_by','Dogs_present', 'Hazardous_materials_present', 
    'Biohazards_present','Size_of_encampment', 'Maptaxlot_hyphen', 'Owner_name', 'Owner_address', 'Nearby_owner', 
    'Nearby_owner_address', 'Ownercity', 'Ownerprvst', 'Ownerzip', 'Geocity_name', 'Ugb_name','Longitude', 'Latitude']]
    taxlotcodes = pd.read_csv('mythical_taxlot_codes.csv')
    for idx in range(0, dat.shape[0]):
        taxlotcode = int(dat.loc[idx,'Maptaxlot_hyphen'][-2:])
        if (str(dat.loc[idx, 'Owner_name']) == 'nan') & (taxlotcode in taxlotcodes.end_number.values):
            dat.loc[idx, 'Owner_name'] = taxlotcodes.loc[taxlotcodes.end_number == taxlotcode, 'taxlot'].values[0].capitalize() + ' R/W'
else:
    k = 0

dat.rename(columns={'Target_fid': 'Target_FID'}, inplace=True)

#intakepath = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\RiparianEcosystemMarketplace\market_area\REM_area.gdb'
shppath = path + '\\MyProject4.gdb'
intake_areas = gpd.read_file(shppath, driver='FileGDB', layer='AboveIntake_Dissolve')
points = gpd.read_file(shppath, driver='FileGDB', layer='HomelessCampSite_SpatialJoin')
points = points.to_crs(epsg=2914)
print("Read data...")

urlstart ='https://services5.arcgis.com/9s1YtFmLS0YTl10F/arcgis/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer/0/'
urlend = '/attachments?f=html&token='

datestr = points.Date.values[0].split('T')[0]
res = convert_date(datestr)
Y = res[1]
m = res[2]
d = res[3]
outfolder = os.path.join(outpath, Y, m+'_'+d)
filename = 'IllegalCampNotice_'+m+'_'+d+'_'+Y[2:4]+'.xlsx'
file = os.path.join(outfolder, filename)
print("Got file name...")

Read data...
Got file name...


In [3]:
for pID in points.index:
    point = points[points.index==pID]
    if all(intake_areas.contains(point)):
        dat.loc[pID, 'Above_Intake'] = 'Yes'
    else:
        dat.loc[pID, 'Above_Intake'] = 'No'
    FID = dat.loc[pID, 'Target_FID']
    url= urlstart+str(FID)+urlend
    resp=requests.get(url)
    links = []
    if resp.status_code==200:
        soup=BeautifulSoup(resp.text,'html.parser')
        for link in soup.findAll('a'):
            links.append(link.get('href'))
    else:
        print("Error")
    attached = [link for link in links if 'attachments' in link]
    if len(attached)==0:
        dat.loc[pID, 'Photos'] = 'NA'
    elif len(attached)==1:
        dat.loc[pID, 'Photos'] = 'https://services5.arcgis.com' + attached[0]
    else:
        dat.loc[pID, 'Photos'] = '; '.join(['https://services5.arcgis.com' + s for s in attached])

print("Edited data...")

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


Edited data...
